In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
install = pd.read_csv('./Indus_OS_Participants_Data/app_installs.csv')
install['uid'] = install['uid'].astype(str)
install['item_id'] = install['item_id'].astype(str)

validation = pd.read_csv('./Indus_OS_Participants_Data/validation_data.csv')
validation['uid'] = validation['uid'].astype(str)

meta = pd.read_csv('./Indus_OS_Participants_Data/app_metadata.csv')
meta['item_id'] = meta['item_id'].astype(str)

# Popular Apps

In [ ]:
fp = pd.read_csv("./Indus_OS_Participants_Data/app_installs.csv")
items = fp.item_id.unique()
dic = {}
for i in items:
    k = fp[fp['item_id'] == i]
    dic[i] = [
        k[k['status'] == 'installed'].shape[0],k[k['status'] == 'uninstalled'].shape[0]
    ]
pop = pd.DataFrame({'id':dic.keys(),'instals':[i[0] for i in dic.values()],'uinstals':[i[1] for i in dic.values()], })


pop['instals'] = pop['instals'] +1
pop['uinstals'] = pop['uinstals'] +1
pop.head()
pop.to_csv('./cont_matrix/popp.csv',index=False)

In [ ]:
pops = pd.read_csv('./cont_matrix/popp.csv')
pops['id'] = pops['id'].astype(str)
ff = dict(pops[['id','score']].values)
def get_fg(x):
    try:
        return ff[x]
    except Exception as e:
        return 0.0

In [ ]:
meta['score'] = meta['item_id'].apply(lambda x: get_fg(x))

In [ ]:
ff = list(set(validation['uid'].astype(str)))
install = install[install['uid'].astype(str).isin(ff)]

In [ ]:
print(len((set(install['item_id'].astype(str))).difference(set(meta['item_id'].astype(str)))))

# creating user's profile

In [ ]:
# install['install_date'] = pd.to_datetime(install['install_date'],infer_datetime_format=True)
df1 = install.sort_values('install_date').groupby('uid',sort=False)['item_id'].apply(list)

In [ ]:
df1 = df1.reset_index()

In [ ]:
categories = dict(meta[['item_id','category']].values)

In [ ]:
def retrieve_cat(x):
    ff =  []
    for xi in x:
        try:
            ff.append(categories[xi])
        except Exception as e:
            ff.append(xi)
    return ff

In [ ]:
df1['categories'] = df1['item_id'].apply(lambda x: retrieve_cat(x))

In [ ]:
# last 8 categories
df1['search_cat'] = df1['categories'].apply(lambda x : x[-8:])

In [ ]:
from statistics import mode
def retrieve_most(xi):
    try:
        x1  = mode(xi)
        if len(xi) < 2:
            return xi
        xi = [i for i in xi if i!=x1]
        x2 = mode(xi)
        return [x1,x2]
    except Exception as e:
        return xi


In [ ]:
# 2 most watched and 5 recent
df1['most_used'] = df1['categories'].apply(lambda x: retrieve_most(x))

In [ ]:
df1['search_category'] = df1['search_cat'] + df1['most_used']

In [ ]:
df1['search_category'] = df1['search_category'].apply(lambda x: list(set(x)))

In [ ]:
df1 = df1[['uid','search_category','item_id']]

In [ ]:
df1.head()

# Most Similar Apps based on popularity, recent and most used app category

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import string
import nltk
from bs4 import BeautifulSoup
import re

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)


In [ ]:
def preprocess(text):
    text = re.sub(r'https\S','',text)
    text = text.lower()
    text = re.sub("\d+", " ", text)
    text = re.sub(r'<br>','',text)
    text = re.sub(r'&amp','',text)
    text = re.sub(r'[:;,-?]',' ',text)
    text = re.sub(r'\n',' ',text)
    text = re.sub('\(.*?\)','',text)
    text = emoji_pattern.sub(r'', text)
    
    return text


In [ ]:
def compute_cosine_similarity(text1, text2):
    
    # stores text in a list
    list_text = [text1, text2]
    
    # converts text into vectors with the TF-IDF 
    vectorizer = TfidfVectorizer(stop_words='english')
    vectorizer.fit_transform(list_text)
    tfidf_text1, tfidf_text2 = vectorizer.transform([list_text[0]]), vectorizer.transform([list_text[1]])
    
    # computes the cosine similarity
    cs_score = cosine_similarity(tfidf_text1, tfidf_text2)
    
    return np.round(cs_score[0][0],2)

In [ ]:
cout = 0
def compare_seneteces(c):
    global cout
    cout +=1
    print(cout)
    # c = df1.iloc[0]
    try:
        items = meta['item_id'].unique()
        used = list(c['item_id'])
        recent = used[-8:]
        recentdesc = meta[meta['item_id'].isin(recent)]['description']
        search_cat = c['search_category']
        
        # meta[(meta['item_id'].isin(left)) & (meta['category'].isin(search_cat))]
        retrieved =  meta[~(meta['item_id'].isin(used))]
        retrieved = retrieved[retrieved['category'].isin(search_cat)][:30]
        
        recent = recentdesc.to_string()
        sen1 = preprocess(recent)
        hash_map = {
        }
        for i in range(len(retrieved)):
        
            k = retrieved.iloc[i]
            sen2 = preprocess(k['description'])
            hash_map[k['item_id']] = compute_cosine_similarity(sen1,sen2)
        hash_map = dict(sorted(hash_map.items(), key=lambda item: item[1],reverse=True))
        return list(hash_map.keys())[:4]
    except Exception as e:
        return None

In [ ]:
df1['recomms']  = df1.apply(lambda x: compare_seneteces(x),axis=1)

In [ ]:
df1 =  df1.drop(['search_category','item_id'],axis=1)

In [ ]:
print(len(set(validation['uid'])))
print(len(set(df1['uid']).intersection(set(validation['uid']))))

In [ ]:
df1 = df1.explode('recomms')

In [ ]:
df1.to_csv('sub1_1.csv',index=False)